In [1]:
import math
import numpy as np
import pandas as pd
import os
import re
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.svm import OneClassSVM

from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

from joblib import dump, load

from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import confusion_matrix

from keras.models import Model
from keras.layers import Input, LSTM, RepeatVector, TimeDistributed, Dense

import tensorflow as tf
from tensorflow.keras import layers, models

import torch
import torch.nn as nn
import torch.optim as optim

from torchsummary import summary

In [3]:
sns.set_theme()
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# Carregando os dados

In [2]:
df_benign = pd.read_csv(("data/dados benignos/logs tratados/Treated_Benign_Traffic.csv"))
df_benign

,Timestamp,Rel Time,DLC,Arb ID,Byte 0,Byte 1,Byte 2,Byte 3
0,0.455000,0.000000,1,165,100,0,0,0
1,0.456296,0.001296,1,165,100,0,0,0
2,0.457849,0.001552,1,165,100,0,0,0
3,0.457961,0.000112,1,165,100,0,0,0
4,0.458867,0.000906,1,165,100,0,0,0
...,...,...,...,...,...,...,...,...
833108,7.490121,0.000167,4,192,230,80,1,86
833109,7.490233,0.000112,1,165,20,0,0,0
833110,7.490400,0.000167,4,192,230,80,1,86
833111,7.490554,0.000154,3,193,190,0,0,0


In [ ]:
df_malicious = pd.read_csv("data/ataques/dados_aleatorios_1.csv")
df_malicious

# Tratando dados

## Ordenando dados e descartando coluna

In [4]:
df_benign = df_benign.sort_values('Timestamp')

df_benign.drop('Timestamp', axis=1, inplace=True)

df_benign

,Rel Time,DLC,Arb ID,Byte 0,Byte 1,Byte 2,Byte 3
505444,0.000000,1,165,99,0,0,0
505445,0.000112,1,165,99,0,0,0
505446,0.000131,2,194,236,255,0,0
505447,0.000132,2,194,236,255,0,0
505448,0.000169,4,192,124,81,1,86
...,...,...,...,...,...,...,...
177951,0.103549,3,193,0,0,0,0
177952,0.000134,2,194,0,0,0,0
177953,0.045736,4,192,0,0,0,0
177954,0.076754,3,193,0,0,0,0


## Dividindo dados em Treino, Validação e Teste

In [5]:
n_linhas = len(df_benign)

# Dividindo o DataFrame em três partes iguais, mas mantendo a ordem dos elementos
df_train = df_benign.iloc[:int(n_linhas*0.6)]   
df_val = df_benign.iloc[int(n_linhas*0.6):int(n_linhas*0.7)]
df_test = df_benign.iloc[int(n_linhas*0.7):]

In [6]:
del df_benign

## Normalização dos dados

In [7]:
minmax_scaler = MinMaxScaler()
minmax_scaler = minmax_scaler.fit(df_train)

norm_df_train = pd.DataFrame(minmax_scaler.transform(df_train), columns=df_train.columns, index=df_train.index)
norm_df_val = pd.DataFrame(minmax_scaler.transform(df_val), columns=df_val.columns, index=df_val.index)
norm_df_test = pd.DataFrame(minmax_scaler.transform(df_test), columns=df_test.columns, index=df_test.index)

In [8]:
del df_train
del df_val
del df_test

## Criando Janelas Deslizantes

# IAs